# Mining Massive Datasets Problem Set 5

Ruben Hartenstein, Taha Erkoc

# Exercise 1

When two different keys hash to the same index in a hash table, we have a hash collision. There are several common mechanisms that are used to resolve hash collisions:

1. Chaining:
Each index inside the hash table stores a linked list of all keys that hash to that index. If a collision occurs, the key-value pair is added to the linked list at that index.

2. Open Adressing:
If a collision occurs, an algorithm probes the hash table for the next available slot (e.g. using linear where it checks the next slot sequentially or quadratic probing where quadratic offsets are used to find the next slot). Unlike in chaining, there is no additional memory needed for linked lists, even though the overall performance can suffer when the table becomes densely populated.

3. Open Hashing (Separate Hash Tables):
Each index inside the hash table stores its own hash table to resolve collisions. This can reduce clustering issues but leads to increased memory usage.


C++, Java and Python all have their own ways to resolve hash collisions.

- C++ (Standard Template Library `std::unordered_map`): Chaining, it uses a bucket array where each bucket contains a linked list or dynamic array of the key-value pairs. Whenever a hash collision occurs, the new key-value pair is added to the bucket corresponding to the hash value. If the load factor (number of elements / number of buckets) surpasses a certain threshold (usually 1), the hash table doubles the size of the bucket arrays and rehashes all its elements.

- Java (`HashMap`): Chaining with Dynamic Resizing, also uses a bucket array where each bucket initially contains a linked list of the key-value pairs. When the number of entries in the bucket surpasses a certain threshold, the linked list in a bucket is converted to a binary search tree for better performance.
Again, after a certain load factor is exceeded, the hash table resizes and all entries are rehashed.

- Python(`dict`): Open adressing with quadratic probing, when a collision occurs, it probes subsequent slots using a quadratic sequence until an empty slot is found or the key is located. Python dictionaries dynamically resize when the table reaches 2/3 of its capacity to maintain performance and ensure that space remains available for probes. During the resizing, all keys are rehashed into the new table.



# Exercise 2

### a)
MurmurHash is a fast, non-crypthographic hash function that primarily uses multiplication and rotation (thus the name Murmur) to achieve a uniform distribution of the hash values. It works by mixing the input data with mathematical operations such as multiplications and XORs to reduce clustering and ensure good randomness of the hash values.

The seed value adds randomization to the hash functions and thus allows the same input string generating different hash outputs when different seeds are used. Using this property it is possible to create independent hash functions which is essential for algorithms like our Bloom filters.


### b)
MurmurHash was never designed to be cryptographically secure thus it just lacks the necessary mathematical complexity to prevent collision attacks. As mentioned above, its main applications are hash tables and Bloom filters, which prioritize speed and a uniform distribition over collision resistance.

# Exercise 3

In [1]:
import pandas as pd
# Stream of integers
stream = [3, 1, 4, 1, 5, 9, 2, 6, 5]

# Hash functions
def h1(x):
    return (2 * x + 1) % 32

def h2(x):
    return (3 * x + 7) % 32

def h3(x):
    return (4 * x) % 32

# Function to count trailing zeros in binary
def count_trailing_zeros(num):
    binary = f"{num:05b}"  # Convert to 5-bit binary
    return len(binary) - len(binary.rstrip('0'))  # Count trailing zeros

# Process each hash function
results = []
hash_functions = [
    (h1, "h1(x) = (2x + 1) mod 32"),
    (h2, "h2(x) = (3x + 7) mod 32"),
    (h3, "h3(x) = (4x) mod 32"),
]

# Iterate through hash functions
for h_func, description in hash_functions:
    hashed_values = [h_func(x) for x in stream]
    tail_lengths = [count_trailing_zeros(h_func(x)) for x in stream]
    max_tail_length = max(tail_lengths)
    estimate = 2 ** max_tail_length
    results.append({
        "Hash Function": description,
        "Hashed Values": hashed_values,
        "Binary Representations": [f"{x:05b}" for x in hashed_values],
        "Tail Lengths": tail_lengths,
        "Max Tail Length": max_tail_length,
        "Estimate (Distinct Elements)": estimate,
    })

# Display results
df_results = pd.DataFrame(results)
print(df_results)


             Hash Function                        Hashed Values  \
0  h1(x) = (2x + 1) mod 32      [7, 3, 9, 3, 11, 19, 5, 13, 11]   
1  h2(x) = (3x + 7) mod 32  [16, 10, 19, 10, 22, 2, 13, 25, 22]   
2      h3(x) = (4x) mod 32     [12, 4, 16, 4, 20, 4, 8, 24, 20]   

                              Binary Representations  \
0  [00111, 00011, 01001, 00011, 01011, 10011, 001...   
1  [10000, 01010, 10011, 01010, 10110, 00010, 011...   
2  [01100, 00100, 10000, 00100, 10100, 00100, 010...   

                  Tail Lengths  Max Tail Length  Estimate (Distinct Elements)  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0]                0                             1  
1  [4, 1, 0, 1, 1, 1, 0, 0, 1]                4                            16  
2  [2, 2, 4, 2, 2, 2, 3, 3, 2]                4                            16  


### Bonus

The choice of hash functions might lead to poor distributions of hash values. For example $h_3(x) = (4x)\mod 32$ will always produce values that are multiples of 4, which significantly reduces randomness and results in fewer unique hash values. This can lead to clustering and many collisions.

An advice would be to avoid simple linear functions like $ax+b\mod 2^k$ unless $a$ and $b$ are carefully chosen to avoid patterns. One could also consider using more advanced hash functions like MurmurHash because these tend to have way better randomness.

# Exercise 4

Given our stream of numbers:

$\{3, 4, 1, 3, 4, 2, 1, 2\}$

First we define our variables $X_i.el$

$X_i.el = [3, 4, 1, 3, 4, 2, 1, 2]$ for $i > 0$ 

The Alon-Matias-Szegedy algorithm computes the $k$-th moment by using the frequency of each unique element in the stream. The variable $X_i.val$ is updated at each position to capture how the stream contributes to the computation of the moment.

So we:
1. Track the frequency of each element $x$ in the stream so far
2. Calculate $X_i.val = f(x)^k$ for the element $x$ at position $i$

For this exercise we take $k = 2$, which is the sum of squared frequencies of the elements.

| Position $i$ | $X_i.el$ | Frequency of $X_i.el$ | $X_i.val = f(X_i.el)^2$|
|--------------|----------|-----------------------|------------------------|
| 1            | 3        | $f(3) = 1$            | $1^2 = 1$              |
| 2            | 4        | $f(4) = 1$            | $1^2 = 1$              |
| 3            | 1        | $f(1) = 1$            | $1^2 = 1$              |
| 4            | 3        | $f(3) = 2$            | $2^2 = 4$              |
| 5            | 4        | $f(4) = 2$            | $2^2 = 4$              |
| 6            | 2        | $f(2) = 1$            | $1^2 = 1$              |
| 7            | 1        | $f(1) = 2$            | $2^2 = 4$              |
| 8            | 2        | $f(2) = 2$            | $2^2 = 4$              |

### Does it make sense to have a separate variable for each stream position?

In practice, the stream is potentially infinite and allocating memory for each position is impossible. Furthermore, many variables would store redundant information, as the streams often depend on statistics (e.g. frequency count in this example).

Instead of storing $X_i.el$ and $X_i.val$ for each position, it would make much more sense to use a frequency dictionary, which tracks the frequency of each unique element in the stream and then use this dictionary to dynamically calculate $X_i.val$ when needed.

# Exercise 5

### a)
The probability formula for a random element hashing to a specific bit in the Bloom filter is: $P(hit) = \frac{1}{n}$

Where $n = 5$ (number of bits in the array). Thus, the probability that a random element gets hashed to a given bit is: $P(hit) = \frac{1}{5} = 20\%$.

This probability arises because each bit in the bit array is equally likely to be chosen by the hash function of a random element, assuming the hash function distributes values uniformly.

### b)
$h_1(x) = x\ mod\ 5$<br>
$h_2(x) = (2x + 3)\ mod\ 5$


For $x = 4$:


$h_1(4) = 4\ mod\ 5 = 4$<br>
$h_2(4) = 11\ mod\ 5 = 1$<br>

|  0   |  1   |  2   |  3   |  4   |
|-----|-----|-----|-----|-----|
| 0 | 1 | 0 | 0 | 1 |


For $x = 1$:


$h_1(4) = 1\ mod\ 5 = 1$<br>
$h_2(4) = 5\ mod\ 5 = 0$<br>

|  0   |  1   |  2   |  3   |  4   |
|-----|-----|-----|-----|-----|
| 1 | 1 | 0 | 0 | 1 |

Every bit is equally likely to be hit by the two hash functions if the input values are uniformly distributed.
- $h_1(x)$ simply maps $x$ to its remainder when divided by 5. Since it cylcles through all possible values as $x$ increases, it uniformly distributes across the 5 bits if $x$ itself is uniformly distributed.
- $h_2(x)$ also distributes the values uniformly since the GCD of 2 and 5 is 1 (2 is relatively prime to 5 since they're both prime numbers). This property ensures that the mapping $2x\ mod\ 5$ generates a complete cycle over all residues before repeating. Adding 3 doesn't change the uniformity but merely shifts the sequence cyclically.


### c)
$P_{false\ positive} = (1 - e^{-\frac{k*n}{m}})^k$

Where:
- $k = 2$ (number of hash functions)
- $n = 2$ (number of elements inserted)
- $m = 5$ (size of the bit array)

The fraction of 1s in the bit array is determined by $1 - e^{-\frac{k*n}{m}} = 0,551$. Thus, approximaltey $55,1\%$ of the bits are expected to be set, resulting in a $P_{false\ positive} = (0,551)^2 = 0,304$.

The false positive probability is approximatley $30,4\%$. This means that for any random number checked against the Bloom filter, there is about a $30,4\%$ chance it will falsely appear to be in the set.

# Exercise 6

# Exercise 7

For a Bloom filter, the false positive probability $b$ is given by:

$b = (1 - e^{-\frac{k*m}{n}})^k$

We now want to find $k$ such that it minimizes $b$, which means we minimize $b$ with respect to $k$.


To do that, we first, take the natural logarithm

$\ln(b) = k * \ln(1 - e^{-\frac{k*m}{n}})$

Now we differentiate $\ln(b)$ with respect to $k$

<br />

$\frac{d}{dk}\ln(b) = \ln(1 - e^{-\frac{k*m}{n}}) + k * \frac{1}{1 - e^{-\frac{k*m}{n}}} * (-\frac{m}{n} * e^{-\frac{k*m}{n}})$

<br />

By setting this derivative to 0 and solving numerically/anatically, the optimal $k$ becomes:

$k = \frac{n}{m} * ln(2)$

This $k$ minimizes collisions while maximizing the effective use of bits.

# Exercise 8

The data has the schema:

$Grades(university, courseID, studentID, grade)$

### a) For each university, the average number of students in a course

Key attributes = `university`, `courseID`

The number of students in a course depends on the courses within each university, therefore we need to sample __universities__ and __courseID__ to ensure that all courses and students within a university are included in the sample.


### b) Fraction of students, who have an average grade of 2.0 or better

Key attributes = `university`, `studentID`

We need to sample the students across all universities to approximate their average grade. By sampling on __studentID__ and __university__, we get a uniform sample of the students where each student is uniquely identified within the sample.

### c) Fraction of courses where at least half of the students got the grade 1.7 or better
Key attributes = `university`, `courseID`

The sampling must preserve the student to course mapping. By sampling on `courseID`, we ensure all students in a course are included in the sample.